In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import gc 
#查看文件列表
import os
print(os.listdir("./input"))
datadir = './input'

['40dims.npy', 'age_test.csv', 'age_train.csv', 'app_info.csv', 'app_info_duplicates.csv', 'df_all.csv', 'merge_age_basic_info_app_actived_test.csv', 'merge_age_basic_info_app_actived_train.csv', 'user_app_actived.csv', 'user_app_actived_40dims.csv', 'user_app_actived_first2.csv', 'user_app_actived_label.csv', 'user_app_usage.csv', 'user_app_usage20190301.csv', 'user_app_usage_aver_time.csv', 'user_app_usage_label_sum0324.csv', 'user_basic_info.csv', 'user_behavior_info.csv']


In [ ]:
user_app_actived = pd.read_csv(os.path.join(datadir,'user_app_actived.csv'), header=None,names=['uId','appId'],dtype={'uId': np.str})
user_app_actived.head()

In [ ]:
user_app_actived.shape

In [12]:
user_app_actived = pd.read_csv(os.path.join(datadir,'user_app_actived.csv'), header=None, names=['uId','appId'],
                            iterator=True, low_memory=False)
for i in range(30):
        try:
                temp = user_app_actived.get_chunk(100000)
                print(i)
                df1=temp.drop(columns=['appId'],axis=1)
                df1=df1.join(temp['appId'].str.split('#',expand=True).stack().reset_index(level=1,drop=True).rename('appId'))
                df1.to_csv(os.path.join('./output','uId_appId_OneByOne.csv'), mode='a',index=False,header=False)
        except StopIteration:
                print('ending..........')
                break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
ending..........


In [ ]:
s=['appId',]
df1=user_app_actived.drop(columns=s,axis=1)
for x in s:
    df1=df1.join(user_app_actived[x].str.split('#',expand=True).stack().reset_index(level=1,drop=True).rename(x))
df1=df1.reset_index(drop=True)

In [19]:
#uId_appId_OneByOne
uId_appId_OneByOne = pd.read_csv(os.path.join('./output','uId_appId_OneByOne.csv'), header=None, 
                                 names=['uId','appId'],iterator=True, low_memory=False)
for i in range(1):
        try:
                temp = uId_appId_OneByOne.get_chunk(100000)
        except StopIteration:
                print('ending..........')
                break
                
temp['category_label'] = LabelEncoder().fit(temp.appId).transform(temp.appId)
temp.head()

,uId,appId,category_label
0,1000008,a00105157,63
1,1000008,a00109386,129
2,1000008,a00134746,493
3,1000008,a0013952,561
4,1000008,a00145168,697


In [2]:
uId_appId_OneByOne = pd.read_csv(os.path.join('./output','uId_appId_OneByOne.csv')
                                 , header=None
                                 ,names=['uId','appId']
                                 ,dtype={'uId': np.str}
                                 ,index_col = 'uId'
                                )
uId_appId_OneByOne['category_label'] = LabelEncoder().fit(uId_appId_OneByOne.appId).transform(uId_appId_OneByOne.appId)
uId_appId_OneByOne.head()

,appId,category_label
uId,,
1000008,a00105157,115
1000008,a00109386,216
1000008,a00134746,781
1000008,a0013952,884
1000008,a00145168,1066


In [14]:
uId_appId_OneByOne.shape

(101803186, 2)

In [3]:
df_train = pd.read_csv(os.path.join(datadir,'age_train.csv')
                       ,header=None
                       , names=['uId','age_group']
                       ,dtype = {'uId':np.str}
                       ,index_col = 'uId')
#df_train.head()
df_test = pd.read_csv(os.path.join(datadir,'age_test.csv')
                      ,names=['uId']
                      ,dtype = {'uId':np.str}
                      ,index_col = 'uId')
df_test.head()

""
uId
1000002
1000003
1000004
1000005
1000006


In [4]:
df_train['trainrow'] = np.arange(df_train.shape[0])
df_test['testrow'] = np.arange(df_test.shape[0])

In [5]:
df_train.head()

,age_group,trainrow
uId,,
1000001,4,0
1000011,3,1
1000015,5,2
1000019,3,3
1000023,2,4


In [6]:
df_test.head()

,testrow
uId,
1000002,0
1000003,1
1000004,2
1000005,3
1000006,4


In [7]:
uId_app = uId_appId_OneByOne.merge(df_train,on='uId')
uId_app.head(100)

,appId,category_label,age_group,trainrow
uId,,,,
1000011,a00158535,1475,3,1
1000011,a00163116,1670,3,1
1000011,a00170432,1880,3,1
1000011,a00187480,2293,3,1
1000011,a00224590,3386,3,1
1000011,a00247519,4062,3,1
1000011,a00287085,5159,3,1
1000011,a00289791,5269,3,1
1000011,a00289826,5270,3,1


In [8]:
uId_app.drop(columns=['appId'],inplace=True)

In [9]:
uId_app.head(100)

,category_label,age_group,trainrow
uId,,,
1000011,1475,3,1
1000011,1670,3,1
1000011,1880,3,1
1000011,2293,3,1
1000011,3386,3,1
1000011,4062,3,1
1000011,5159,3,1
1000011,5269,3,1
1000011,5270,3,1


In [15]:
uId_app.shape

(81433320, 3)

In [16]:
#d = deviceapps.dropna(subset=['trainrow'])
Xtr_app = csr_matrix((np.ones(uId_app.shape[0]), (uId_app.trainrow, uId_app.category_label)), 
                      shape=(df_train.shape[0],9401))
#d = deviceapps.dropna(subset=['testrow'])
#Xte_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
#                      shape=(gatest.shape[0],napps))
Xtr_app.shape
#print('Apps data: train shape {}, test shape {}'.format(Xtr_app.shape, Xte_app.shape))

(2010000, 9401)

In [17]:
Xtr_app

<2010000x9401 sparse matrix of type '<class 'numpy.float64'>'
	with 81433320 stored elements in Compressed Sparse Row format>

In [18]:
uId_app_test = uId_appId_OneByOne.merge(df_test,on='uId')
uId_app_test.head(100)

,appId,category_label,testrow
uId,,,
1000008,a00105157,115,6
1000008,a00109386,216,6
1000008,a00134746,781,6
1000008,a0013952,884,6
1000008,a00145168,1066,6
1000008,a00157201,1404,6
1000008,a00162045,1630,6
1000008,a0016381,1689,6
1000008,a00170432,1880,6


In [19]:
uId_app_test.drop(columns=['appId'],inplace=True)
uId_app_test.head(100)

,category_label,testrow
uId,,
1000008,115,6
1000008,216,6
1000008,781,6
1000008,884,6
1000008,1066,6
1000008,1404,6
1000008,1630,6
1000008,1689,6
1000008,1880,6


In [21]:
uId_app_test.isnull().sum()

category_label    0
testrow           0
dtype: int64

In [22]:
Xte_app = csr_matrix((np.ones(uId_app_test.shape[0]), (uId_app_test.testrow, uId_app_test.category_label)), 
                      shape=(df_test.shape[0],9401))

In [24]:
Xtr_app.shape,Xte_app.shape

((2010000, 9401), (502500, 9401))

In [25]:
#保存特征
from scipy import sparse
sparse.save_npz('Xtr_app9401.npz', Xtr_app)  #保存
sparse.save_npz('Xte_app9401.npz', Xte_app)  #保存

In [4]:
uId_appId_OneByOne.appId.nunique()

9401

In [5]:
uId_appId_OneByOne.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
uId,101803186,2512500,3164162,888,NaN,NaN,NaN,NaN,NaN,NaN,NaN
appId,101803186,9401,a00289791,2339048,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_label,1.01803e+08,NaN,NaN,NaN,4987.15,2398.25,0,3375,5215,6752,9400
